<a href="https://colab.research.google.com/github/mmistroni/TensorFlowPlayground/blob/master/PySpark_Colab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h3> Installing dependencies</h3>

In [0]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://www-us.apache.org/dist/spark/spark-2.4.4/spark-2.4.4-bin-hadoop2.7.tgz
!tar xf spark-2.4.4-bin-hadoop2.7.tgz
!pip install -q findspark

In [0]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.4-bin-hadoop2.7"

In [0]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

<p> ToDO test few runs here, then  store a dataframe of all possible shares for last 2 years and run a correlation between all cols </b>

In [0]:
# Use this link to calculate correlation https://stackoverflow.com/questions/45112976/how-to-use-correlation-in-spark-with-dataframes

In [21]:
!pip install pandas-datareader

In [0]:
import requests
import pandas_datareader.data as dr
from datetime import date
def get_historical_value(symbol):
  try: 
    data = dr.get_data_yahoo(symbol, date(2018,1,1), date(2019,9,19))[['Adj Close']]
    return data.rename(columns={'Adj Close' : symbol})[symbol].values.tolist()
    
  except Exception as e :
    print('Excepiton for {}:{}'.format(symbol, str(e)))
    return []
  
def get_all_stocks_data():
  all_symbols_data = requests.get('https://cloud.iexapis.com/stable/ref-data/iex/symbols?token=sk_98e397d4bee940488e1f48e9b419508f').json()
  good_ones = [d['symbol'] for d in all_symbols_data if d['isEnabled']][0:20]
  return filter(lambda data: bool(data), map(lambda symbol: get_historical_value(symbol), good_ones))


In [0]:
all_stocks  = get_all_stocks_data()


In [0]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.stat import Correlation


In [0]:
data = [(Vectors.dense(stock_data),) for stock_data in all_stocks]

In [30]:
len(data)

20

In [31]:
df = spark.createDataFrame(data, ["features"])

r1 = Correlation.corr(df, "features").head()
print("Pearson correlation matrix:\n" + str(r1[0]))

Py4JJavaError: ignored